This notebook will be used for the Applied Data Science Capstone Project

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [8]:
#Store the URL for the wikipedia page
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Get the HTML text for the URL
resp = requests.get(URL)
html_code = resp.text

In [16]:
#Instantiate html parser object
soup = BeautifulSoup(html_code)

#Get the first table on the webpage and all rows within it 
table = soup.find_all('table')[0]
cells = table.find_all('p')

In [17]:

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for c in cells:    
    #Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if  c.span.text != 'Not assigned' :
        postalcode = c.b.text
        borough = (c.span.text.replace('(',',').replace(')','').replace('/',','))
        borough = (re.split((','), borough))
        df = df.append({"PostalCode":postalcode, "Borough":borough[0], "Neighborhood":borough[1:]}, ignore_index=True)


In [23]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
3,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
4,M7A,Queen's Park,[Ontario Provincial Government]


In [22]:
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.lstrip('[').rstrip(']'))

AttributeError: 'list' object has no attribute 'lstrip'

In [21]:
df['Borough'].iloc[0:5]

0          North York
1          North York
2    Downtown Toronto
3          North York
4        Queen's Park
Name: Borough, dtype: object

In [24]:
!pip install pgeocode
import pgeocode
nomi = pgeocode.Nominatim('ca')

In [69]:
latitude = []
longitude = []
for i in range(len(df['PostalCode'])):
  coord= nomi.query_postal_code(df['PostalCode'].iloc[i])
  latitude.append(coord.latitude)
  longitude.append(coord.longitude)

df['Latitude'] = latitude
df['Longitude'] = longitude

In [70]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.7545,-79.3300
1,M4A,North York,[Victoria Village],43.7276,-79.3148
2,M5A,Downtown Toronto,"[Regent Park , Harbourfront]",43.6555,-79.3626
3,M6A,North York,"[Lawrence Manor , Lawrence Heights]",43.7223,-79.4504
4,M7A,Queen's Park,[Ontario Provincial Government],43.6641,-79.3889


In [71]:
df.dropna(axis=0, inplace=True)

In [72]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [73]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
tor_lat = location.latitude
tor_lon  = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_lon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [74]:
#create Toronto dataset
df_toronto = df[df['Borough'].str.contains("Toronto")]
df_toronto['Neighborhood'] = df_toronto['Neighborhood'].astype(str) 
print(df_toronto.shape)
df_toronto.head()


(39, 5)
<ipython-input-74-e31d687b8bc8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toronto['Neighborhood'] = df_toronto['Neighborhood'].astype(str)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626
9,M5B,Downtown Toronto,"['Garden District', ' Ryerson']",43.6572,-79.3783
15,M5C,Downtown Toronto,['St. James Town'],43.6513,-79.3756
19,M4E,East Toronto,['The Beaches'],43.6784,-79.2941
20,M5E,Downtown Toronto,['Berczy Park'],43.6456,-79.3754


In [75]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [76]:
CLIENT_ID = 'I3C1JCMVYJ5C0RGHF2VVQH31BZY4EOOFNPZ2Z1OM1SUFHAV0' # your Foursquare ID
CLIENT_SECRET = 'VJ3WUI2BTMYCV0EKJNOBUDSMQ43DUFALJYXNJPQPVXF4NSZW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [77]:
neighborhood_latitude = df_toronto['Latitude'].iloc[0] # neighborhood latitude value
neighborhood_longitude = df_toronto['Longitude'].iloc[0] # neighborhood longitude value

neighborhood_name = df_toronto['Neighborhood'].iloc[0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Regent Park ', ' Harbourfront'] are 43.6555, -79.3626.


In [80]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results



y',
          'lat': 43.65691857501867,
          'lng': -79.35896684476664}],
        'distance': 332,
        'postalCode': 'M5A 1T9',
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['500 Queen Street East',
         'Toronto ON M5A 1T9',
         'Canada']},
       'categories': [{'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'shortName': 'Pub',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-566e1294498e3f6629006bc3-6'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae5b91ff964a520a6a121e3',
       'name': 'Morning Glory Cafe',
       'location': {'ad

In [81]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [82]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-82-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Souvlaki Express,Greek Restaurant,43.655584,-79.364438
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


In [83]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [86]:
# type your answer here
toronto_venues = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'])

['Regent Park ', ' Harbourfront']
['Garden District', ' Ryerson']
['St. James Town']
['The Beaches']
['Berczy Park']
['Central Bay Street']
['Christie']
['Richmond ', ' Adelaide ', ' King']
['Dufferin ', ' Dovercourt Village']
['The Danforth  East']
['Harbourfront East ', ' Union Station ', ' Toronto Islands']
['Little Portugal ', ' Trinity']
['The Danforth West ', ' Riverdale']
['Toronto Dominion Centre ', ' Design Exchange']
['Brockton ', ' Parkdale Village ', ' Exhibition Place']
['India Bazaar ', ' The Beaches West']
['Commerce Court ', ' Victoria Hotel']
['Studio District']
['Lawrence Park']
['Roselawn']
['Davisville North']
['Forest Hill North & West']
['High Park ', ' The Junction South']
['North Toronto West']
['The Annex ', ' North Midtown ', ' Yorkville']
['Parkdale ', ' Roncesvalles']
['Davisville']
['University of Toronto ', ' Harbord']
['Runnymede ', ' Swansea']
['Moore Park ', ' Summerhill East']
['Kensington Market ', ' Chinatown ', ' Grange Park']
['Summerhill West ', '

In [92]:
print(toronto_venues.shape)
toronto_venues.head()

(1507, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,"['Regent Park ', ' Harbourfront']",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [93]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
['Berczy Park'],94,94,94,94,94,94
"['Brockton ', ' Parkdale Village ', ' Exhibition Place']",40,40,40,40,40,40
"['CN Tower ', ' King and Spadina ', ' Railway Lands ', ' Harbourfront West ', ' Bathurst Quay ', ' South Niagara ', ' Island airport']",58,58,58,58,58,58
['Central Bay Street'],60,60,60,60,60,60
['Christie'],12,12,12,12,12,12
['Church and Wellesley'],78,78,78,78,78,78
"['Commerce Court ', ' Victoria Hotel']",100,100,100,100,100,100
['Davisville North'],7,7,7,7,7,7
['Davisville'],23,23,23,23,23,23


Analyze each Neighborhood

In [95]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"['Regent Park ', ' Harbourfront']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"['Regent Park ', ' Harbourfront']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,